In [ ]:
using ESDL
using WeightedOnlineStats

In this Notebook we want to visualize the regions where cube variable take on their highest or lowest value. To do this we calculate the 99% and the 1% quantile for each variable from a subset of the data and afterwards count how often the quantile was exceeded for each variable in each grid cell.

In [ ]:
c             = Cube()
vars          = ["air_temperature_2m","soil_moisture","gross_primary_productivity"];
cdata         = subsetcube(c,region="South America",variable=vars,time=2001:2002)

Although it might seem to be a trivial task to estimate a quantile of some datacube variables, it actually isn't. For an exact quantile estimate, the whole dataset would need to be in memory. However, it is possible to get aproximate estimates of the quantiles by first fitting a `WeightedHistogram` provided by the [WeightedOnlineStats](https://github.com/gdkrmr/WeightedOnlineStats.jl) package and estimate the quantiles based on the Histogram. We first generate an iterable Table from the cube data:

In [ ]:
t = CubeTable(cdata = cdata, include_axes=("lat","var"))

This is a row iterator over the cube which will load data on demand. To fit a `WeightedHist` split by variable and weighted by cosine-latitude weights we use the convenience function `cubefittable` provided by the ESDL framework:

In [ ]:
o = cubefittable(t,WeightedAdaptiveHist(200),:cdata,by=(:var,), weight=i->cosd(i.lat))

In [ ]:
using ESDLPlots
plotHist(o,var="soil_moisture")

This datacube encodes a weighted Histogram for every variable with 100 bins. We can estimate the 1% and 99% quantiles from the Histogram:

In [ ]:
q = quantile(o,[0.01,0.99])

Now we can use the estimated quantiles to count for each pixel how often the quantile has been exceeded. TO do this, we define a function that counts quantile crossings for each grid cell:

In [ ]:
"""
How often the upper or lower quantiles are crossed in each time series
"""
function countExtremes(xout::AbstractArray,xin::AbstractVector,qvec::AbstractVector)
    nlow,nhigh=0,0
    qlow,qhigh=qvec
    for v in xin
        if !ismissing(v)
            v<=qlow && (nlow+=1)
            v>=qhigh && (nhigh+=1)
        end
    end
    xout[1]=nhigh
    xout[2]=nlow
end

In [ ]:
indims = InDims("Time"),InDims("Quantile")
outdims = OutDims(CategoricalAxis("Direction",["High","Low"]),outtype=Int)
tresexcount=mapCube(countExtremes,(cdata,q),
  indims = indims,
  outdims = outdims)

Now we plot the low extremes

In [ ]:
for v in getAxis("Var",o).values
display(v)
display(plotMAP(tresexcount,dmax=10,dir="High",var=v))
end

And the high extremes

In [ ]:
for v in getAxis("Var",o).values
display(v)
display(plotMAP(tresexcount,dmax=10,dir="Low",var=v))
end